### 3개 이미지 선택 -> 분석 -> 키워드 나오게

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# 모델 로드
model_path = "TL_inception.h5"  # 모델 파일 경로 설정
loaded_model = load_model(model_path)  # load_model은 모델을 불러오는 함수입니다. 이전에 모델을 저장했어야 합니다.

# 이미지 경로 설정
image_paths = ["./crawled_img/sample/4.jpg", "./crawled_img/sample/2.jpeg", "./crawled_img/sample/1.jpg"]

# 카테고리 리스트
categories =["joyful", "adventure", "tradition", "nature", "cultural", "art"]

# 이미지 분류 함수
def classify_image(model, img_path, categories):
    # 이미지 로드 및 전처리
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # 이미지를 0과 1 사이 값으로 정규화

    # 이미지 예측
    predictions = model.predict(img_array)

    # 예측 결과에서 가장 높은 확률을 가진 클래스 선택
    predicted_class = np.argmax(predictions)

    # 카테고리 반환
    return categories[predicted_class]

# 각 이미지에 대해 분류 실행
predicted_results = []
for img_path in image_paths:
    result = classify_image(loaded_model, img_path, categories)
    predicted_results.append(result)

# 중복된 결과 제거
unique_results = list(set(predicted_results))

# 예측 결과 출력
for i, img_path in enumerate(image_paths):
    print(f"Image {i+1}: {img_path}")
print("Predicted Results:")
for result in unique_results:
    print(f"- {result}")

1/1 [==============================] - 0s 108ms/step
Image 1: ./crawled_img/sample/4.jpg
Image 2: ./crawled_img/sample/2.jpeg
Image 3: ./crawled_img/sample/1.jpg
Predicted Results:
- nature
- tradition
- adventure


# 한개 이미지 업로드 -> 분석 -> 3개의 키워드

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# 모델 불러오기 (학습된 모델 파일을 지정해주세요)
model = load_model('TL_inception.h5')

# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]

# 이미지 크기 설정 (학습시 사용한 이미지 크기와 동일해야 함)
img_size = (224, 224)

# 이미지를 불러오고 전처리하는 함수
def preprocess_image(image_path):
    img = load_img(image_path, target_size=img_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # 모델 학습 때 적용한 정규화
    img_array = np.expand_dims(img_array, axis=0)  # 모델 입력을 위해 차원 확장
    return img_array

# 단일 이미지 경로
image_path = "./crawled_img/sample/3.jpg"

# 예측 함수
def predict_top_categories(image_path, top=3):
    # 이미지 전처리
    img_array = preprocess_image(image_path)

    # 예측
    predictions = model.predict(img_array)

    # 상위 3개의 카테고리와 확률을 구함
    top_indices = predictions[0].argsort()[-top:][::-1]
    top_probs = predictions[0][top_indices]
    top_categories = [categories[idx] for idx in top_indices]

    # 예측 결과 출력
    print("Predicted Results")
    for i in range(top):
        print(f"{i+1}. {top_categories[i]} {top_probs[i] * 100:.2f}%")

# 이미지 경로에 대한 예측 수행
predict_top_categories(image_path)


1/1 [==============================] - 2s 2s/step
Predicted Results
1. nature 99.01%
2. tradition 0.81%
3. adventure 0.15%


### 3개 이미지 선택 -> 3개를 종합해서 1개로 분석 -> 키워드 3개 추출

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# 모델 로드
model_path = "TL_inception.h5"  # 모델 파일 경로 설정
loaded_model = load_model(model_path)

# 이미지 경로 설정
image_paths = ["./crawled_img/sample/4.jpg", "./crawled_img/sample/2.jpeg", "./crawled_img/sample/1.jpg"]

# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]

# 이미지 분류 함수
def classify_image(model, img_path):
    # 이미지 로드 및 전처리
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # 이미지를 0과 1 사이 값으로 정규화

    # 이미지 예측
    predictions = model.predict(img_array)

    # 예측 결과에서 가장 높은 확률을 가진 클래스 선택
    predicted_class = np.argmax(predictions)

    return predicted_class, predictions[0]

# 이미지별로 예측 실행
all_predictions = np.zeros((len(image_paths), 6))  # 6은 카테고리 수
for i, img_path in enumerate(image_paths):
    _, predictions = classify_image(loaded_model, img_path)
    all_predictions[i] = predictions

# 이미지 결과 종합
average_predictions = np.mean(all_predictions, axis=0)
top_keywords = np.argsort(average_predictions)[::-1][:3]

# 출력
print("Selected Images:")
for img_path in image_paths:
    print(f"- {img_path}")

print("\nPredicted Keywords:")
for idx in top_keywords:
    percentage = average_predictions[idx] * 100
    print(f"- {categories[idx]}: {percentage:.2f}%")


1/1 [==============================] - 0s 107ms/step
Selected Images:
- ./crawled_img/sample/4.jpg
- ./crawled_img/sample/2.jpeg
- ./crawled_img/sample/1.jpg

Predicted Keywords:
- adventure: 33.36%
- nature: 33.33%
- tradition: 33.22%
